In [75]:
from difflib import SequenceMatcher
import pandas as pd
import re
import os 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [76]:
combo = pd.read_csv('data/combined.csv')
['published', 'title', 'description', 'link', 'Pub']
# print(combo.columns.tolist())

['published', 'title', 'description', 'link', 'Pub']

In [77]:
def search_similar(df,
                   queries: list[str],
                   columns: list[str],
                   threshold: float = 0.6,
                   token_overlap_min: float = 0.5):
    queries = [q.strip().casefold() for q in queries if q and str(q).strip()]
    if not queries:
        return df.iloc[0:0]

    word_re = re.compile(r"\w+")

    def token_overlap(a: str, b: str) -> float:
        sa, sb = set(word_re.findall(a)), set(word_re.findall(b))
        if not sa or not sb:
            return 0.0
        denom = min(len(sa), len(sb))
        return len(sa & sb) / denom

    def is_similar(text) -> bool:
        if pd.isna(text):
            return False
        t = str(text).casefold().strip()
        if not t:
            return False

        for q in queries:
            if q in t or t in q:
                return True
            if token_overlap(t, q) >= token_overlap_min:
                return True
            if SequenceMatcher(None, t, q).ratio() >= threshold:
                return True
        return False

    mask = df[columns].applymap(is_similar).any(axis=1)
    # return df[mask]
    returno = df[mask].copy()
    returno['published'] = pd.to_datetime(returno['published']).dt.tz_convert('Australia/Sydney').dt.strftime("%-I%p %d %b")
    # returno = returno[['title', 'link']]

    for index, row in returno.iterrows():
        print(row['title'])
        print(row['published'], row['link'])
        print("\n")

In [78]:
os.system('python3 youtube.py')

####
terms = ["environment", 'epbc', 'watt']
###

search_similar(combo, terms, [ 'title', 'description'], threshold=0.5)



Barnaby Joyce boycotts Nationals over net zero | 7NEWS
12PM 27 Oct https://www.youtube.com/watch?v=VWXL5S8vhk4


Labor pushes for green light on its overhaul of Australia's environment laws | ABC NEWS
7PM 26 Oct https://www.youtube.com/watch?v=yqlxnbuXb28




Traceback (most recent call last):
  File "/Users/josh_nicholas/Personal/collect-clips/youtube.py", line 4, in <module>
    import feedparser
ModuleNotFoundError: No module named 'feedparser'
